## Load datasets

In [1]:
import pandas as pd
import numpy as np

import tasks.postprocessing
import tasks.constants


main_df = tasks.postprocessing.get_main_dataset()
main_df = main_df.iloc[:, [0, 1, 9]]
main_df = main_df.drop_duplicates("message_id")
main_df["ablation_feature"] = "all"
main_df["ablation_factor"] = "original"
main_df

,conv_id,message_id,message,ablation_feature,ablation_factor
0,3bd075d6-b91b-4f60-9138-c33feff56d2c,876311276352101160,How do you know he is not proud of his cakes? ...,all,original
11,3bd075d6-b91b-4f60-9138-c33feff56d2c,-1870930993328192561,"@SkepticalInvestor77, you're missing the point...",all,original
22,3bd075d6-b91b-4f60-9138-c33feff56d2c,-1583336252148959998,"@CitySlicker05, thank you for clarifying your ...",all,original
33,3bd075d6-b91b-4f60-9138-c33feff56d2c,838628372202429525,"@SkepticalInvestor77, I understand your perspe...",all,original
44,3bd075d6-b91b-4f60-9138-c33feff56d2c,1852804548951377081,"@GentleTherapist56, thank you for your thought...",all,original
...,...,...,...,...,...
57266,8fd69250-71ad-4b24-9dca-0658d6eb95fc,1972079638350772560,"@WanderlustNomad22, thank you for bringing up ...",all,original
57277,8fd69250-71ad-4b24-9dca-0658d6eb95fc,1183976761754781031,I completely agree that addressing systemic is...,all,original
57288,8fd69250-71ad-4b24-9dca-0658d6eb95fc,1769825593077317243,"@NatureLover88, thank you for sharing your per...",all,original
57299,8fd69250-71ad-4b24-9dca-0658d6eb95fc,1394900428090388415,"@NatureLover88, I totally agree with you about...",all,original


In [2]:
abl_df = tasks.postprocessing.get_ablation_df()
abl_df

,conv_id,message_id,message,ablation_feature,ablation_factor
0,e9001267-fba5-4e81-9340-10256439f896,-618891699449759534,How do you know he is not proud of his cakes? ...,prompts,noroles
1,e9001267-fba5-4e81-9340-10256439f896,-300876236281035132,"@SurvivalistHunter65 Okay, first off, let’s be...",prompts,noroles
2,e9001267-fba5-4e81-9340-10256439f896,-136774720061539955,NaN,prompts,noroles
3,e9001267-fba5-4e81-9340-10256439f896,-170339060548748661,"@SurvivalistHunter65, @RebelliousTeen44, let's...",prompts,noroles
4,e9001267-fba5-4e81-9340-10256439f896,-90638422715427188,"@RebelliousTeen44, @OptimisticNurse42 Thanks f...",prompts,noroles
...,...,...,...,...,...
1461,86f6b244-2f63-40f1-87dc-11eb6e4e0862,1269456098477038129,NaN,turns,random
1462,86f6b244-2f63-40f1-87dc-11eb6e4e0862,2099748626367877414,@MysteriousArtist34 That's beautiful. Art and ...,turns,random
1463,86f6b244-2f63-40f1-87dc-11eb6e4e0862,1269456098477038129,NaN,turns,random
1464,86f6b244-2f63-40f1-87dc-11eb6e4e0862,-1492233235653878353,"@NatureLover88 Absolutely, I couldn't agree mo...",turns,random


Data from CeRI http://archive.regulationroom.org/

Any opinions, findings, and conclusions or recommendations expressed in this material are those of the author(s) and do not necessarily reflect the views of the CeRI (Cornell e-Rulemaking Initiative).

In [3]:
human_df = tasks.postprocessing.get_human_df()
human_df

,conv_id,message_id,message,ablation_feature,ablation_factor
0,What counts as texting,249.0,You probably ought to explicitly call out the ...,all,human
1,What are the risks?,250.0,I would be interested in knowing why the DOT d...,all,human
2,What counts as texting,251.0,I don't believe that entering a phone number o...,all,human
3,What are the risks?,252.0,"Personally, I don't think there should be a la...",all,human
4,What are the risks?,253.0,\t\t\t\t_x000D_\t\t\t\tWelcome to Regulation R...,all,human
...,...,...,...,...,...
3644,"Roads & bridges: Repair, restoration, moderniz...",288.0,<p>Just because the Move NY FAIR Plan has the ...,all,human
3645,Bus: More express bus and Select Bus service; ...,289.0,<p>The proposed SBS project for Woodhaven Blvd...,all,human
3646,Dealing with Transportation Deserts: Full-feat...,290.0,<p>Thanks for letting us know - we fixed the m...,all,human
3647,"Roads & bridges: Repair, restoration, moderniz...",291.0,"<p>Hi rickhoran, Welcome to the discussion and...",all,human


In [4]:
df = pd.concat([main_df, abl_df, human_df], ignore_index=True)
df

,conv_id,message_id,message,ablation_feature,ablation_factor
0,3bd075d6-b91b-4f60-9138-c33feff56d2c,876311276352101160,How do you know he is not proud of his cakes? ...,all,original
1,3bd075d6-b91b-4f60-9138-c33feff56d2c,-1870930993328192561,"@SkepticalInvestor77, you're missing the point...",all,original
2,3bd075d6-b91b-4f60-9138-c33feff56d2c,-1583336252148959998,"@CitySlicker05, thank you for clarifying your ...",all,original
3,3bd075d6-b91b-4f60-9138-c33feff56d2c,838628372202429525,"@SkepticalInvestor77, I understand your perspe...",all,original
4,3bd075d6-b91b-4f60-9138-c33feff56d2c,1852804548951377081,"@GentleTherapist56, thank you for your thought...",all,original
...,...,...,...,...,...
8681,"Roads & bridges: Repair, restoration, moderniz...",288.0,<p>Just because the Move NY FAIR Plan has the ...,all,human
8682,Bus: More express bus and Select Bus service; ...,289.0,<p>The proposed SBS project for Woodhaven Blvd...,all,human
8683,Dealing with Transportation Deserts: Full-feat...,290.0,<p>Thanks for letting us know - we fixed the m...,all,human
8684,"Roads & bridges: Repair, restoration, moderniz...",291.0,"<p>Hi rickhoran, Welcome to the discussion and...",all,human


In [5]:
df.message = df.message.astype(str)
df.message = np.where(df.message == "nan", "", df.message)
df

,conv_id,message_id,message,ablation_feature,ablation_factor
0,3bd075d6-b91b-4f60-9138-c33feff56d2c,876311276352101160,How do you know he is not proud of his cakes? ...,all,original
1,3bd075d6-b91b-4f60-9138-c33feff56d2c,-1870930993328192561,"@SkepticalInvestor77, you're missing the point...",all,original
2,3bd075d6-b91b-4f60-9138-c33feff56d2c,-1583336252148959998,"@CitySlicker05, thank you for clarifying your ...",all,original
3,3bd075d6-b91b-4f60-9138-c33feff56d2c,838628372202429525,"@SkepticalInvestor77, I understand your perspe...",all,original
4,3bd075d6-b91b-4f60-9138-c33feff56d2c,1852804548951377081,"@GentleTherapist56, thank you for your thought...",all,original
...,...,...,...,...,...
8681,"Roads & bridges: Repair, restoration, moderniz...",288.0,<p>Just because the Move NY FAIR Plan has the ...,all,human
8682,Bus: More express bus and Select Bus service; ...,289.0,<p>The proposed SBS project for Woodhaven Blvd...,all,human
8683,Dealing with Transportation Deserts: Full-feat...,290.0,<p>Thanks for letting us know - we fixed the m...,all,human
8684,"Roads & bridges: Repair, restoration, moderniz...",291.0,"<p>Hi rickhoran, Welcome to the discussion and...",all,human


### Comment length

In [6]:
len_df = df.copy()
len_df["comment_length"] = len_df.message.apply(lambda x: len(x.split()))
len_df = len_df.loc[
    :, ["message_id", "comment_length", "ablation_factor"]
]
len_df

,message_id,comment_length,ablation_factor
0,876311276352101160,179,original
1,-1870930993328192561,78,original
2,-1583336252148959998,28,original
3,838628372202429525,196,original
4,1852804548951377081,40,original
...,...,...,...
8681,288.0,35,human
8682,289.0,83,human
8683,290.0,13,human
8684,291.0,71,human


In [7]:
sns.histplot(len_df, x="comment_length", hue="model", kde=True)
plt.xlabel("Comment Length (#words)")

tasks.graphs.save_plot(GRAPH_OUTPUT_DIR / "comment_length.png")
plt.show()

NameError: name 'sns' is not defined

In [ ]:
message_df = df.copy()
message_df = message_df.drop_duplicates(subset=["conv_id", "message_id"])
# @ tokens crash bleu scorer
message_df.message = message_df.message.apply(
    lambda msg: " ".join(word for word in msg.split() if not word.startswith("@"))
)

similarity_df = (
    message_df.groupby(["conv_id", "model"])["message"]
    .apply(lambda messages: messages.tolist())
    .reset_index()
)
similarity_df.columns = ["conv_id", "model", "messages"]
similarity_df = similarity_df[similarity_df.model != "hardcoded"]

In [ ]:
similarity_df["rougel_similarity"] = similarity_df.messages.progress_apply(
    tasks.stats.pairwise_rougel_similarity
)

In [ ]:
sns.histplot(similarity_df, x="rougel_similarity", hue="model", kde=True)
plt.xlabel("Average pairwise F1 ROUGE-L")
plt.title("Intra-discussion comment variance")

tasks.graphs.save_plot(GRAPH_OUTPUT_DIR / "discussion_variance.png")
plt.show()